## Mood map

In [ ]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import ast

In [ ]:
# center the map around north-east of Italy
box = [45.65, 11.75] # lat, long

# create the map
my_map = folium.Map(location = box, zoom_start = 9)

In [ ]:
df = pd.read_csv('mood_map.csv')
df = df.drop(["Unnamed: 0"], axis=1)
df.head()

In [ ]:
for row in range(len(df)):
    if df.places_dict[row].startswith('{'):
        pl = ast.literal_eval(df.places_dict[row])
        all_values = pl.values()
        max_value = max(all_values)
        for k in pl.keys():
            if pl[k] == max_value:
                df.at[row, 'place'] = k   
    else:
        pl = df.places_dict[row]
        df.at[row, 'place'] = pl

df = df.drop(["places_dict"], axis=1)
df.head()

In [ ]:
# mood function to associate emoticon to value of avg.mood
def mood_function(avg_mood):
    if avg_mood <2: # sad & very sad
        return "<img src='https://cdn-icons-png.flaticon.com/512/927/927561.png' width='20'/>"
    elif avg_mood >= 2 and avg_mood < 2.6: # neutral
        return "<img src='https://cdn-icons-png.flaticon.com/512/927/927557.png' width='20'/>"
    else: # avg_mood >= 3 --> happy & very happy
        return "<img src='https://cdn-icons-png.flaticon.com/512/1023/1023656.png' width='20'/>"

In [ ]:
df.frequency.unique()

In [ ]:
print(df.avg_mood.min(), df.avg_mood.max())

In [ ]:
'''
"Frequency" column reports the (absolute) counting of the number of times the location have been visited in the two weeks of
data collection.
Considering the short period of time, here it is assumed that if the place is found more than 10 times, then it is an important
location wrt the subject, thus highlighted in the map with the green marker.
'''
def color_marker(f): 
    if f < 5:
        return 'red'
    elif f >= 5 and f <= 10:
        return 'orange'
    else: # f > 10
        return 'green'

In [ ]:
# check all possible activities
s = set()
for row in range(len(df)):
    for el in ast.literal_eval(df.what_lst[row]):
        s.add(el)
s

In [ ]:
# top-3 of the most done activities by location
import ast
from collections import Counter
for row in range(len(df)):
    what_list = ast.literal_eval(df.what_lst[row])
    what_dict_act_counts = Counter(what_list) # convert what_list into a dictionary having as keys the activities and as values the num of time that activity appear in the list
    # getting top-3 keys having highest value (highest num of counts for that activity in this specific location)
    most_common = sorted(what_dict_act_counts, key=what_dict_act_counts.get, reverse=True)[:3] # first el in the list is the top-1 activity
    df.at[row, 'Top3'] = most_common
    
for row in range(len(df)):
    what_list = ast.literal_eval(df.what_lst[row])
    what_dict_act_counts = Counter(what_list) # convert what_list into a dictionary having as keys the activities and as values the num of time that activity appear in the list
    # getting top-3 keys having highest value (highest num of counts for that activity in this specific location)
    # most_common = sorted(what_dict_act_counts, key=what_dict_act_counts.get, reverse=True)[:3] # first el in the list is the top-1 activity
    most_common = what_dict_act_counts.most_common(3) # top-3 with also the countings
    df.at[row, 'Top3'] = most_common
df

In [ ]:
# not used
def top3_activities_by_place(row):
    import matplotlib.pyplot as plt; plt.rcdefaults()
    import numpy as np
    import matplotlib.pyplot as plt
    
    #for row in range(len(df)):
    x = []
    ticks_act = []
    for el in range(len(df.Top3[row])):
        x.append(df.Top3[row][el][1])
        ticks_act.append(df.Top3[row][el][0])
    y = np.arange(len(df.Top3[row]))
    
    ticks = []
    for word in ticks_act:
        label = word.split(' ')
        ticks.append(' '.join(l + '\n' * (n % 3 == 2) for n, l in enumerate(label)))
        
    ticks_y = []
    for el in range(0, len(df.Top3[row])+2):
        ticks_y.append(el)
        
    plt.bar(y, x, align='center', alpha=0.5)
    plt.xticks(y, ticks, rotation=0, horizontalalignment='center')
    plt.yticks(ticks_y)
    plt.ylabel('')
    plt.title('Activities')

    plt.show()
        
top3_activities_by_place(3)

In [ ]:
def top3_activities(row):
    act = []
    for el in range(len(df.Top3[row])):
        act.append(df.Top3[row][el][0])
    a = ''
    for word in range(len(act)):
        label = act[word].split(' ')
        a = a + '<strong>' + str(word+1) + '. </strong>' + ' '.join(label) + '<br>'
    return a

top3_activities(3)

In [ ]:
# cut address
for row in range(len(df)):
    lst = df.City[row].split(',')
    # address
    s = lst[0] + ',' + lst[1]  
    # adding city
    if lst[-6] not in s:
        s = s + ',' + lst[-6]
    elif lst[-5] not in s:
        s = s + ',' + lst[-5]
    # change value in df
    df.at[row, 'City'] = s

In [ ]:
# legend
import branca

legend_html = '''
{% macro html(this, kwargs) %}
<div style="
    position: fixed; 
    bottom: 50px;
    left: 50px;
    width: 250px;
    height: 110px;
    z-index:9999;
    font-size:14px;
    ">
    <p style="color:black;font-size:100%;margin-left:10px;">Importance (frequency)</p>
    <p><a style="color:red;font-size:150%;margin-left:20px;">&#9632;</a>&emsp;Less than 5 times</p>
    <p><a style="color:orange;font-size:150%;margin-left:20px;">&#9632;</a>&emsp;From 5 to 10 times</p>
    <p><a style="color:green;font-size:150%;margin-left:20px;">&#9632;</a>&emsp;More than 10 times</p>
</div>
<div style="
    position: fixed; 
    bottom: 10px;
    left: 50px;
    width: 200px;
    height: 160px; 
    z-index:9998;
    font-size:14px;
    background-color: #ffffff;

    opacity: 0.7;
    ">
</div>
{% endmacro %}
'''
legend = branca.element.MacroElement()
legend._template = branca.element.Template(legend_html)

In [ ]:
# center the map around north-east of Italy
box = [45.65, 11.75] # lat, long

# create the map
my_map = folium.Map(location = box, zoom_start = 9)


for row in range(len(df)): 
    pop_up = folium.Html(mood_function(df.avg_mood[row]) + '<br>' + str(round(df.avg_mood[row],2)) + '<br>' + '<strong>Address: </strong>' + str(df.City[row]) +
                         '<br> <strong>Subjective place: </strong>' + str(df.place[row]) + '<br><br><strong>' + 
                         'Top-3 activities by location:</strong>' + '<br>' + top3_activities(row), script=True)
    popup = folium.Popup(pop_up, max_width=2650)
    folium.Marker([df.lat[row], df.long[row]], popup = popup,
                 icon=folium.Icon(color = color_marker(df.frequency[row]), icon_color = 'white')).add_to(my_map)

my_map.get_root().add_child(legend) # add legend
my_map


In [ ]:
# my_map.save('mood_map.html')

In [ ]:
# zoom 
box = [46.1, 11.1] # lat, long

my_map = folium.Map(location = box, zoom_start = 15)

for row in range(len(df)):
    pop_up = folium.Html(mood_function(df.avg_mood[row]) + '<br>' + str(round(df.avg_mood[row],2)) + '<br>' + '<strong>Address: </strong>' + str(df.City[row]) +
                         '<br> <strong>Subjective place: </strong>' + str(df.place[row]) + '<br><br><strong>' + 
                         'Top-3 activities by location:</strong>' + '<br>' + top3_activities(row), script=True)
    popup = folium.Popup(pop_up, max_width=2650)
    folium.Marker([df.lat[row], df.long[row]], popup = popup,
                 icon=folium.Icon(color = color_marker(df.frequency[row]), icon_color = 'white')).add_to(my_map)

    
my_map.get_root().add_child(legend)
my_map

In [ ]:
# zoom
box = [45.5, 12.8]

my_map = folium.Map(location = box, zoom_start = 14)

for row in range(len(df)):
    pop_up = folium.Html(mood_function(df.avg_mood[row]) + '<br>' + str(round(df.avg_mood[row],2)) + '<br>' + '<strong>Address: </strong>' + str(df.City[row]) +
                         '<br> <strong>Subjective place: </strong>' + str(df.place[row]) + '<br><br><strong>' + 
                         'Top-3 activities by location:</strong>' + '<br>' + top3_activities(row), script=True)
    popup = folium.Popup(pop_up, max_width=2650)
    folium.Marker([df.lat[row], df.long[row]], popup = popup,
                 icon=folium.Icon(color = color_marker(df.frequency[row]), icon_color = 'white')).add_to(my_map)
    

my_map.get_root().add_child(legend)
my_map

In [ ]:
poi_df = df[df['frequency'] > 10] # only green points (PoI)
poi_df